# Download the datafile [here](https://www.dropbox.com/s/0r508fymxtnepuz/all_data.hdf?dl=0) and place in the same folder as this notebook.

In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
df = pd.read_hdf('all_data.hdf')
d = SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                    phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                    I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                    I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                    B=hv.Dimension('$B$', unit='T', values=df['B_x'].unique().tolist()),
                    T=hv.Dimension('$T$', unit='K', values=df['T'].unique().tolist()), 
                    L=hv.Dimension('$L$', unit='nm', values=df['L'].unique().tolist()), 
                    g=hv.Dimension('$g$-factor', values=df['g'].unique().tolist()), 
                    alpha=hv.Dimension(r'$\alpha$', unit='mev nm', values=df['alpha'].unique().tolist()), 
                    mu=hv.Dimension(r'$\mu$', unit='meV', values=df['mu'].unique().tolist()), 
                    a=hv.Dimension('$a$', unit='nm', values=df['a'].unique().tolist()))

pd.set_option('display.max_columns', None)

# Critical current and phase, $B_x$ vs $I_c$ and $B_x$ vs $\phi_c$

In [ ]:
inds = ['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = [d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]
gb = df.groupby(inds)
groups = dict(list(gb))

def current_curve(T, L, leads, orbital, g, alpha, mu, a):
    gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
    return hv.Path((gr['B_x'], gr['current_c']), kdims=[d.B, d.I_c])[:, 0:df['current_c'].max()]

def phase_curve(T, L, leads, orbital, g, alpha, mu, a):
    gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
    return hv.Path((gr['B_x'], gr['phase_c']), kdims=[d.B, d.phi_c])[:, -np.pi:np.pi]

kdims = [d.T, d.L, hv.Dimension('leads', values=[True, False]), 
         hv.Dimension('orbital', values=[True, False]),
         d.g, d.alpha, d.mu, d.a]

hv.DynamicMap(current_curve, kdims=kdims) + hv.DynamicMap(phase_curve, kdims=kdims)

# Current phase relation, $\phi$ vs $I$
Adjust `zoom` slider to change the `y`-range of the plot.

In [ ]:
%%opts Path [aspect='square']
inds = ['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = [d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]
gb = df.groupby(inds)
groups = dict(list(gb))

def current_curve(B_x, T, L, leads, orbital, g, alpha, mu, a, zoom):
    gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
    gr = gr[gr['B_x'] == B_x]
    I_max = df['current_c'].max()
    return hv.Path((np.array(gr['phases'])[0], np.array(gr['currents'])[0]), kdims=[d.B, d.I_c])[:, -zoom:zoom]

kdims = [d.B, d.T, d.L, hv.Dimension('leads', values=[True, False]), 
         hv.Dimension('orbital', values=[True, False]), d.g, d.alpha, 
         d.mu, d.a, hv.Dimension('zoom', values=[10, 20, 40, 80, 160, 200])]

hv.DynamicMap(current_curve, kdims=kdims)